In [ ]:
from pywinauto import application
import time
import os

# 증권사 API 자동 실행
os.system('taskkill /IM coStarter* /F /T')
os.system('taskkill /IM CpStart* /F /T')
os.system('wmic process where "name like \'%coStarter%\'" call terminate')
os.system('wmic process where "name like \'%CpStart%\'" call terminate')
time.sleep(5)        

app = application.Application()
app.start('C:\CREON\STARTER\coStarter.exe /prj:cp /id:아이디 /pwd:비밀번호 /pwdcert:인증서 비밀번호 /autostart')
time.sleep(60)

In [1]:
import win32com.client
import ctypes

# 서버 접속 확인
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect

def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False

    if (bConnect == 1): #접속 - 1, 비접속 - 0
        print("정상: 서버에 접속되었습니다.")
    else:
        print("오류: 서버에 연결되지 않았습니다. ")
        return False

InitPlusCheck()

정상: 관리자권한으로 실행된 프로세스입니다.
정상: 서버에 접속되었습니다.


In [41]:
import win32com.client
import pandas as pd
from datetime import datetime
import time

# 오브젝트 가져오기
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')

objRq = win32com.client.Dispatch("CpSysDib.CpSvr7210d")

objRq.SetInputValue(0, '0')
# 0 전체 1 거래소 2 코스닥 3 업종 4 관심종목
objRq.SetInputValue(1, ord('0'))
# 0 수량 1 금액
objRq.SetInputValue(2, 0)
# 0 종목 1 외국인 2 기관계 3 보험기타 4 투신..
objRq.SetInputValue(3, ord('0'))
# 0 상위순 1 하위순

sumcnt = 0
df = None
df = pd.DataFrame(columns=('code', '종목명', '현재가', '대비', '대비율', '거래량', '외국인', '기관계', '보험기타금융', '투신', '은행', '연기금', '사모펀드', '기타법인'))

while True:

    remainCount = g_objCpStatus.GetLimitRemainCount(1)  # 1 시세 제한

    if remainCount <= 0:
        print('시세 연속 조회 제한 회피를 위해 sleep', g_objCpStatus.LimitRequestRemainTime)
        time.sleep(g_objCpStatus.LimitRequestRemainTime / 1000)

    objRq.BlockRequest()
 
    cnt = objRq.GetHeaderValue(0)
    sumcnt += cnt
    #date = objRq.GetHeaderValue(1) # 집계날짜
    #time = objRq.GetHeaderValue(2)  # 집계시간
    print(cnt)
 
    for i in range(cnt):
        item = {}
        item['code'] = objRq.GetDataValue(0, i)
        item['종목명'] = objRq.GetDataValue(1, i)
        item['현재가'] = objRq.GetDataValue(2, i)
        item['대비'] = objRq.GetDataValue(3, i)
        item['대비율'] = objRq.GetDataValue(4, i)
        item['거래량'] = objRq.GetDataValue(5, i)
        item['외국인'] = objRq.GetDataValue(6, i)
        item['기관계'] = objRq.GetDataValue(7, i)
        item['보험기타금융'] = objRq.GetDataValue(8, i)
        item['투신'] = objRq.GetDataValue(9, i)
        item['은행'] = objRq.GetDataValue(10, i)
        item['연기금'] = objRq.GetDataValue(11, i)
        item['국가지자체'] = objRq.GetDataValue(12, i)
        item['기타법인'] = objRq.GetDataValue(13, i)

        df.loc[len(df)] = item

        # 1000개 처리
    if sumcnt > 3000:
        break
    # 연속 처리
    if objRq.Continue != True:
        break
print(123)

600
600
600
600
600
199
123


In [40]:
df

,code,종목명,현재가,대비,대비율,거래량,외국인,기관계,보험기타금융,투신,은행,연기금,사모펀드,기타법인
0,A036630,세종텔레콤,809,33,4.25,52650979,1893297,0,0,0,0,0,NaN,82000
1,A038880,아이에이,1520,-60,-3.80,17073048,808845,0,0,0,0,0,NaN,-1000
2,A035250,강원랜드,26100,1100,4.40,3412176,759731,167000,3000,139000,15000,10000,NaN,-3000
3,A105560,KB금융,45800,300,0.66,2850177,639449,-169000,2000,-83000,0,-88000,NaN,28000
4,A055550,신한지주,34050,400,1.19,3495041,604676,-90000,101000,-130000,0,-61000,NaN,43000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,A375500,DL이앤씨,111500,500,0.45,378574,-92561,-5000,0,-5000,0,0,NaN,2000
1183,A285130,SK케미칼,315500,-31500,-9.08,601656,-116666,-9000,0,-6000,0,-3000,NaN,-1000
1184,A303030,지니틱스,3300,-110,-3.23,13452956,-130000,0,0,0,0,0,NaN,-94000
1185,A900280,골든센츄리,476,-16,-3.25,44366840,-349000,0,0,0,0,0,NaN,0


In [ ]:
df.to_excel('종목별 투자자 현황.xlsx', encoding='cp949')

In [47]:
import win32com.client
import pandas as pd
from datetime import datetime
import time

# 오브젝트 가져오기
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')

objRq = win32com.client.Dispatch("CpSysDib.MarketEye")

codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
print(codeList)
print(codeList2)
allcodelist = codeList + codeList2

rqField = [0, 5 ,20,118,120] #요청 필드 종목코드 , 시가 , 상장주식수, 당일외국인순매수, 당일기관순매수
rqCodeList = []



sumcnt = 0
df = pd.DataFrame(columns=('code', '종목명', '상장주식수','시가총액','당일외국인순매수','당일기관순매수','외국인순매수비율','기관순매수비율'))

'''
for i, code in enumerate(allcodelist):
    rqCodeList.append(code)
    if len(rqCodeList) == 200:
        print(len(rqCodeList))
        rqCodeList = []
'''

codeindex = 0
allcodeindex = len(allcodelist)
print(allcodeindex)
while True:
    rqCodeList = []
    for i in range(200):
        if allcodeindex <= codeindex + i:
            break
        rqCodeList.append(allcodelist[codeindex + i])
    codeindex += len(rqCodeList)
    
    
    remainCount = g_objCpStatus.GetLimitRemainCount(1)  # 1 시세 제한
    if remainCount <= 0:
        print('시세 연속 조회 제한 회피를 위해 sleep', g_objCpStatus.LimitRequestRemainTime)
        time.sleep(g_objCpStatus.LimitRequestRemainTime / 1000)
 
    objRq.SetInputValue(0, rqField)
    # 0 기본, 0 종목코드 20 총주식수
    objRq.SetInputValue(1, rqCodeList)
    # 1 종목코드 또는 종목코드 배열(최대 200종목까지 가능)

    objRq.BlockRequest()

    cnt = objRq.GetHeaderValue(2)
    sumcnt += cnt
    print(sumcnt)
    for i in range(cnt):
        item = {}
        item['code'] = objRq.GetDataValue(0, i) #종목코드
        curval = objRq.GetDataValue(1, i) #시가
        if curval == 0:
            continue #정지종목은 시가가 0
        item['상장주식수'] = objRq.GetDataValue(2, i) #상장주식수
        item['시가총액'] = item['상장주식수'] * curval #시가총액
        item['당일외국인순매수'] = objRq.GetDataValue(3, i) #당일외국인순매수
        item['당일기관순매수'] = objRq.GetDataValue(4, i) #당일외국인순매수
        item['외국인순매수비율'] = item['당일외국인순매수']  / (item['상장주식수'] / 100000) 
        item['기관순매수비율'] = item['당일기관순매수'] / (item['상장주식수'] / 100000)
        
        
        df.loc[len(df)] = item

    if sumcnt >= allcodeindex:
        break
df

('A000020', 'A000040', 'A000050', 'A000060', 'A000070', 'A000075', 'A000080', 'A000087', 'A000100', 'A000105', 'A000120', 'A000140', 'A000145', 'A000150', 'A000155', 'A000157', 'A000180', 'A000210', 'A000215', 'A000220', 'A000225', 'A000227', 'A000230', 'A000240', 'A000270', 'A000300', 'A000320', 'A000325', 'A000370', 'A000390', 'A000400', 'A000430', 'A000480', 'A000490', 'A000500', 'A000520', 'A000540', 'A000545', 'A000547', 'A000590', 'A000640', 'A000650', 'A000660', 'A000670', 'A000680', 'A000700', 'A000720', 'A000725', 'A000760', 'A000810', 'A000815', 'A000850', 'A000860', 'A000880', 'A000885', 'A00088K', 'A000890', 'A000910', 'A000950', 'A000970', 'A000990', 'A000995', 'A001020', 'A001040', 'A001045', 'A00104K', 'A001060', 'A001065', 'A001067', 'A001070', 'A001080', 'A001120', 'A001130', 'A001140', 'A001200', 'A001210', 'A001230', 'A001250', 'A001260', 'A001270', 'A001275', 'A001290', 'A001340', 'A001360', 'A001380', 'A001390', 'A001420', 'A001430', 'A001440', 'A001450', 'A001460'

200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000


,code,종목명,상장주식수,시가총액,당일외국인순매수,당일기관순매수,외국인순매수비율,기관순매수비율
0,A000020,NaN,27931470,396626874000,345,-197,1.235166,-0.705298
1,A000040,NaN,96137368,117287588960,-24345,0,-25.323140,0.000000
2,A000050,NaN,27415270,344061638500,-3839,464,-14.003145,1.692487
3,A000060,NaN,120625000,2050625000000,-46672,-105554,-38.691813,-87.505907
4,A000070,NaN,8564271,767358681600,-2561,-2766,-29.903304,-32.296970
...,...,...,...,...,...,...,...,...
2861,A340120,NaN,5020000,10441600000,0,2,0.000000,0.039841
2862,A340350,NaN,5710000,11676950000,0,97,0.000000,1.698774
2863,A340440,NaN,4710000,9608400000,0,1,0.000000,0.021231
2864,A340570,NaN,4064000,156464000000,-903,0,-22.219488,0.000000


In [32]:
import win32com.client
import pandas as pd
from datetime import datetime
import time
g_obj7216 = win32com.client.Dispatch('CpSysDib.CpSvrNew7216')
g_obj7216.SetInputValue(0,'A950180')
g_obj7216.BlockRequest()
cnt = g_obj7216.GetHeaderValue(1)
sdate = g_obj7216.GetHeaderValue(2)
print(cnt)
print(sdate)
print('---')
for i in range(cnt):
    print('--')
    print(g_obj7216.GetDataValue(0,i))
    print(g_obj7216.GetDataValue(6,i))
    print(g_obj7216.GetDataValue(8,i))

242
20210304
---
--
20210304
0
0
--
20210303
0
10074
--
20210302
0
-23258
--
20210226
-50
-19623
--
20210225
-2449
39909
--
20210224
2549
49036
--
20210223
-1000
5157
--
20210222
0
569
--
20210219
-329583
-139547
--
20210218
-19346
12039
--
20210217
0
3141
--
20210216
0
140
--
20210215
1000
-13280
--
20210210
0
-905
--
20210209
0
-1027
--
20210208
0
2024
--
20210205
-5096
-7480
--
20210204
0
-17594
--
20210203
0
6701
--
20210202
-1859
3834
--
20210201
4526
9236
--
20210129
3094
-6128
--
20210128
689
8903
--
20210127
0
247
--
20210126
10222
-89608
--
20210125
14728
13085
--
20210122
1927
-26662
--
20210121
12643
22941
--
20210120
10070
5005
--
20210119
18926
-24272
--
20210118
0
10465
--
20210115
-2822
1201
--
20210114
2822
-33082
--
20210113
23699
-3422
--
20210112
0
-34744
--
20210111
-6357
-66213
--
20210108
0
-17848
--
20210107
0
-13593
--
20210106
0
-107319
--
20210105
-8149
-173042
--
20210104
-8648
-59634
--
20201230
0
13454
--
20201229
0
15778
--
20201228
358
-12071
--
20201224


In [34]:
import win32com.client
import pandas as pd
from datetime import datetime
import time

# 오브젝트 가져오기
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')

class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수
        rqField = [0, 4, 20]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
            # print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))

        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print(data2)
 
 
if __name__ == "__main__":
    objMarketTotal = CMarketTotal()
    objMarketTotal.GetAllMarketTotal()
    objMarketTotal.PrintMarketTotal()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[('A005930', (5969782, 491910036800000)), ('A000660', (728002365, 103376335830000)), ('A035420', (164263395, 64309119142500)), ('A051910', (70592343, 61062376695000)), ('A005935', (822886700, 60153017770000)), ('A005380', (213668187, 50212023945000)), ('A207940', (66165000, 49491420000000)), ('A006400', (68764530, 46622351340000)), ('A035720', (88678085, 42742836970000)), ('A068270', (135027731, 41385999551500)), ('A000270', (405363347, 31902095408900)), ('A012330', (94793094, 29433255687000)), ('A005490', (87186835, 26940732015000)), ('A096770', (92465564, 24410908896000)), ('A066570', (163647814, 23892580844000)), ('A051900', (15618197, 23364822712000)), ('A028260', (186887081, 22800223882000)), ('A036570', (21954022, 21075861120000)), ('A017670', (80745711, 19903817761500)), ('A091990', (151971142, 19893022487800)), ('A034730', (70360297, 19208361081000)), ('A105560', (415807920, 19044002736000)), ('A055550', (516599554, 17590214813700)), ('A003550', (172557131, 15875256052000)), (